# 專案實戰: 聊天機器人系統 (Chatbot with Hugging Face)

**專案類型**: 對話系統 - 基於 Transformer 的聊天機器人
**難度**: ⭐⭐⭐⭐ 進階
**預計時間**: 4-5 小時
**技術棧**: Hugging Face Transformers, DialoGPT, Streamlit

---

## 📚 學習目標

完成本專案後,您將能夠:

1. ✅ 理解對話系統的核心架構
2. ✅ 使用 Hugging Face 對話模型 (DialoGPT, Blenderbot)
3. ✅ 實作多輪對話管理
4. ✅ 構建互動式聊天介面
5. ✅ 部署生產級聊天機器人

---

## Part 1: 對話系統基礎

### 1.1 對話系統類型

| 類型 | 說明 | 應用場景 | 範例 |
|------|------|---------|------|
| **檢索式** | 從預定義回覆中選擇 | 客服 FAQ | 選擇最相關答案 |
| **生成式** | 動態生成回覆 | 開放對話 | GPT, DialoGPT |
| **任務導向** | 完成特定任務 | 訂票、查詢 | 槽填充對話 |
| **閒聊型** | 自由對話 | 陪伴機器人 | Blenderbot |

### 1.2 本專案架構

```
用戶輸入
    ↓
文本預處理
    ├── Tokenization
    ├── 添加歷史對話
    └── 構建 Context
    ↓
對話模型 (DialoGPT)
    ├── Encoder: 理解輸入
    ├── Decoder: 生成回覆
    └── Attention: 關注重點
    ↓
後處理
    ├── Beam Search 選擇最佳回覆
    ├── 過濾不當內容
    └── 格式化輸出
    ↓
機器人回覆
```

## Part 2: 環境準備與套件安裝

In [ ]:
# 安裝必要套件
# !pip install transformers torch accelerate streamlit -q

# 驗證安裝
import transformers
import torch

print(f"✅ Transformers 版本: {transformers.__version__}")
print(f"✅ PyTorch 版本: {torch.__version__}")
print(f"✅ CUDA 可用: {torch.cuda.is_available()}")

## Part 3: 基礎聊天機器人實作

### 3.1 使用 DialoGPT (Microsoft)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# 載入 DialoGPT 模型 (3 種規模可選)
model_name = "microsoft/DialoGPT-medium"  # small, medium, large

print(f"載入模型: {model_name}...")
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

print(f"✅ 模型載入完成")
print(f"   參數量: {model.num_parameters():,}")
print(f"   詞彙表大小: {tokenizer.vocab_size:,}")

### 3.2 單輪對話實作

In [ ]:
def generate_response(user_input, chat_history_ids=None, max_length=1000):
    """
    生成聊天機器人回覆

    Args:
        user_input: 用戶輸入文本
        chat_history_ids: 對話歷史 (用於多輪對話)
        max_length: 最大生成長度

    Returns:
        response: 機器人回覆
        new_chat_history_ids: 更新的對話歷史
    """
    # 編碼用戶輸入
    new_input_ids = tokenizer.encode(
        user_input + tokenizer.eos_token,
        return_tensors='pt'
    )

    # 合併對話歷史
    if chat_history_ids is not None:
        bot_input_ids = torch.cat([chat_history_ids, new_input_ids], dim=-1)
    else:
        bot_input_ids = new_input_ids

    # 生成回覆
    chat_history_ids = model.generate(
        bot_input_ids,
        max_length=max_length,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,           # 啟用採樣
        top_k=50,                 # Top-K 採樣
        top_p=0.95,               # Nucleus 採樣
        temperature=0.7           # 控制創造性
    )

    # 解碼回覆
    response = tokenizer.decode(
        chat_history_ids[:, bot_input_ids.shape[-1]:][0],
        skip_special_tokens=True
    )

    return response, chat_history_ids


# 測試單輪對話
user_input = "Hello! How are you?"
response, history = generate_response(user_input)

print(f"User: {user_input}")
print(f"Bot: {response}")

### 3.3 多輪對話實作

In [ ]:
# 多輪對話測試
chat_history_ids = None
conversation = [
    "Hi there!",
    "What's your favorite programming language?",
    "Why do you like it?",
    "Do you know about NLP?",
    "Goodbye!"
]

print("=" * 60)
print("多輪對話示範")
print("=" * 60)

for i, user_input in enumerate(conversation, 1):
    response, chat_history_ids = generate_response(
        user_input,
        chat_history_ids=chat_history_ids
    )

    print(f"\n第 {i} 輪對話:")
    print(f"User: {user_input}")
    print(f"Bot: {response}")
    print("-" * 60)

## Part 4: 進階功能實作

### 4.1 對話歷史管理

In [ ]:
class ChatHistoryManager:
    """
    管理對話歷史，防止記憶體溢出
    """
    def __init__(self, max_history_length=1000):
        self.max_history_length = max_history_length
        self.chat_history_ids = None

    def add_to_history(self, new_input_ids, response_ids):
        """添加對話到歷史"""
        if self.chat_history_ids is None:
            self.chat_history_ids = torch.cat([new_input_ids, response_ids], dim=-1)
        else:
            self.chat_history_ids = torch.cat(
                [self.chat_history_ids, new_input_ids, response_ids],
                dim=-1
            )

        # 限制歷史長度
        if self.chat_history_ids.shape[-1] > self.max_history_length:
            self.chat_history_ids = self.chat_history_ids[:, -self.max_history_length:]

    def get_history(self):
        """獲取當前歷史"""
        return self.chat_history_ids

    def clear_history(self):
        """清空歷史"""
        self.chat_history_ids = None

    def get_conversation_text(self, tokenizer):
        """獲取對話歷史文本"""
        if self.chat_history_ids is None:
            return ""
        return tokenizer.decode(self.chat_history_ids[0], skip_special_tokens=True)


# 使用範例
history_manager = ChatHistoryManager(max_history_length=500)

user_input = "Tell me about Python"
response, history_ids = generate_response(user_input)

print(f"User: {user_input}")
print(f"Bot: {response}")
print(f"\n歷史長度: {history_ids.shape[-1]} tokens")

### 4.2 回覆品質控制

In [ ]:
import re

class ResponseFilter:
    """
    過濾不當回覆,提升對話品質
    """
    def __init__(self):
        # 不當詞彙列表 (示範,實際應更完整)
        self.blocked_words = set(['badword1', 'badword2'])

    def is_valid_response(self, response):
        """
        檢查回覆是否合格
        """
        # 檢查 1: 非空
        if not response or len(response.strip()) == 0:
            return False, "空回覆"

        # 檢查 2: 長度合理
        if len(response) < 3:
            return False, "回覆過短"

        if len(response) > 500:
            return False, "回覆過長"

        # 檢查 3: 無重複 (避免 "I I I I...")
        words = response.split()
        if len(words) > 3 and len(set(words)) < len(words) * 0.3:
            return False, "過度重複"

        # 檢查 4: 無不當詞彙
        if any(word in response.lower() for word in self.blocked_words):
            return False, "包含不當詞彙"

        return True, "合格"

    def clean_response(self, response):
        """
        清理回覆
        """
        # 移除多餘空白
        response = re.sub(r'\s+', ' ', response).strip()

        # 確保句號結尾
        if response and response[-1] not in '.!?':
            response += '.'

        return response


# 測試回覆過濾
filter = ResponseFilter()

test_responses = [
    "I love Python programming!",
    "I I I I I",  # 重複
    "",  # 空回覆
    "Ok"  # 過短
]

for resp in test_responses:
    is_valid, reason = filter.is_valid_response(resp)
    print(f"回覆: '{resp}'")
    print(f"  有效: {is_valid} ({reason})\n")

### 4.3 完整聊天機器人類別

In [ ]:
class Chatbot:
    """
    完整的聊天機器人系統
    """
    def __init__(self, model_name="microsoft/DialoGPT-medium"):
        print(f"初始化聊天機器人: {model_name}")

        # 載入模型
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_pretrained(model_name)

        # 對話歷史
        self.chat_history_ids = None

        # 回覆過濾器
        self.response_filter = ResponseFilter()

        print("✅ 聊天機器人準備完成!")

    def chat(self, user_input, max_retries=3):
        """
        處理用戶輸入並生成回覆
        """
        # 編碼輸入
        new_input_ids = self.tokenizer.encode(
            user_input + self.tokenizer.eos_token,
            return_tensors='pt'
        )

        # 合併歷史
        if self.chat_history_ids is not None:
            bot_input_ids = torch.cat([self.chat_history_ids, new_input_ids], dim=-1)
        else:
            bot_input_ids = new_input_ids

        # 嘗試生成合格回覆 (最多重試 3 次)
        for attempt in range(max_retries):
            # 生成回覆
            chat_history_ids = self.model.generate(
                bot_input_ids,
                max_length=1000,
                pad_token_id=self.tokenizer.eos_token_id,
                do_sample=True,
                top_k=50,
                top_p=0.95,
                temperature=0.7 + (attempt * 0.1)  # 重試時增加溫度
            )

            # 解碼
            response = self.tokenizer.decode(
                chat_history_ids[:, bot_input_ids.shape[-1]:][0],
                skip_special_tokens=True
            )

            # 檢查品質
            is_valid, reason = self.response_filter.is_valid_response(response)

            if is_valid:
                # 更新歷史
                self.chat_history_ids = chat_history_ids

                # 清理回覆
                response = self.response_filter.clean_response(response)

                return response
            else:
                print(f"  嘗試 {attempt+1}: 回覆無效 ({reason})")

        # 所有重試失敗,返回備用回覆
        return "I'm sorry, I didn't quite understand that. Could you rephrase?"

    def reset(self):
        """重置對話歷史"""
        self.chat_history_ids = None
        print("✅ 對話歷史已清空")

    def get_conversation_length(self):
        """獲取對話長度"""
        if self.chat_history_ids is None:
            return 0
        return self.chat_history_ids.shape[-1]


# 創建聊天機器人實例
bot = Chatbot(model_name="microsoft/DialoGPT-medium")

### 4.4 互動式對話介面 (Jupyter Widget)

In [ ]:
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets

class ChatInterface:
    """
    Jupyter Notebook 聊天介面
    """
    def __init__(self, chatbot):
        self.chatbot = chatbot
        self.conversation_history = []

        # UI 元件
        self.output = widgets.Output()
        self.user_input = widgets.Text(
            placeholder='輸入訊息...',
            description='您:',
            layout=widgets.Layout(width='80%')
        )
        self.send_button = widgets.Button(
            description='發送',
            button_style='primary'
        )
        self.reset_button = widgets.Button(
            description='重置對話',
            button_style='warning'
        )

        # 綁定事件
        self.send_button.on_click(self.on_send)
        self.reset_button.on_click(self.on_reset)
        self.user_input.on_submit(self.on_send)

    def on_send(self, b):
        """處理發送事件"""
        user_message = self.user_input.value.strip()

        if not user_message:
            return

        # 清空輸入框
        self.user_input.value = ''

        # 生成回覆
        with self.output:
            print(f"\n👤 You: {user_message}")

        bot_response = self.chatbot.chat(user_message)

        with self.output:
            print(f"🤖 Bot: {bot_response}")
            print("-" * 60)

        # 記錄對話
        self.conversation_history.append({
            'user': user_message,
            'bot': bot_response
        })

    def on_reset(self, b):
        """重置對話"""
        self.chatbot.reset()
        self.conversation_history = []

        with self.output:
            clear_output()
            print("✅ 對話已重置")

    def display(self):
        """顯示聊天介面"""
        input_box = widgets.HBox([self.user_input, self.send_button, self.reset_button])
        chat_box = widgets.VBox([self.output, input_box])

        display(HTML("<h3>💬 聊天機器人介面</h3>"))
        display(chat_box)


# 啟動聊天介面
chat_interface = ChatInterface(bot)
chat_interface.display()

## Part 5: 使用 Blenderbot (Meta)

### 5.1 Blenderbot vs DialoGPT 對比

| 模型 | 開發者 | 規模 | 特色 | 適用場景 |
|------|--------|------|------|----------|
| **DialoGPT** | Microsoft | 117M-762M | 基於 Reddit 訓練 | 開放閒聊 |
| **Blenderbot** | Meta (Facebook) | 90M-9.4B | 多技能整合 | 知識問答+閒聊 |
| **DialoGPT** | 生成速度快 | 較輕量 | 英文對話 |
| **Blenderbot** | 知識豐富 | 較重量 | 多語言支持 |

### 5.2 使用 Blenderbot

In [ ]:
from transformers import BlenderbotTokenizer, BlenderbotForConditionalGeneration

# 載入 Blenderbot
blenderbot_model_name = "facebook/blenderbot-400M-distill"

print(f"載入 Blenderbot: {blenderbot_model_name}")
blenderbot_tokenizer = BlenderbotTokenizer.from_pretrained(blenderbot_model_name)
blenderbot_model = BlenderbotForConditionalGeneration.from_pretrained(blenderbot_model_name)

print("✅ Blenderbot 載入完成")


def chat_with_blenderbot(user_input):
    """
    使用 Blenderbot 對話
    """
    inputs = blenderbot_tokenizer([user_input], return_tensors="pt")

    reply_ids = blenderbot_model.generate(
        **inputs,
        max_length=128,
        num_beams=5,              # Beam Search
        early_stopping=True,
        no_repeat_ngram_size=3   # 避免重複
    )

    response = blenderbot_tokenizer.decode(
        reply_ids[0],
        skip_special_tokens=True
    )

    return response


# 測試 Blenderbot
test_inputs = [
    "What is natural language processing?",
    "Tell me about Python programming.",
    "What's your favorite book?"
]

print("🤖 Blenderbot 對話測試\n")
for inp in test_inputs:
    response = chat_with_blenderbot(inp)
    print(f"User: {inp}")
    print(f"Bot: {response}\n")

## Part 6: 特殊功能擴展

### 6.1 情緒識別

In [ ]:
from transformers import pipeline

# 載入情感分析器
emotion_classifier = pipeline(
    "sentiment-analysis",
    model="j-hartmann/emotion-english-distilroberta-base"
)

def detect_emotion(text):
    """
    檢測用戶情緒
    """
    result = emotion_classifier(text)[0]
    return result['label'], result['score']

def generate_empathetic_response(user_input, emotion):
    """
    根據用戶情緒調整回覆風格
    """
    # 情緒前綴模板
    emotion_templates = {
        'joy': "That's wonderful! ",
        'sadness': "I'm sorry to hear that. ",
        'anger': "I understand your frustration. ",
        'fear': "Don't worry, ",
        'surprise': "Oh! "
    }

    # 獲取基本回覆
    base_response = bot.chat(user_input)

    # 添加情緒前綴
    prefix = emotion_templates.get(emotion.lower(), "")
    empathetic_response = prefix + base_response

    return empathetic_response


# 測試情緒感知對話
test_messages = [
    "I just got a promotion at work!",
    "I'm feeling really sad today.",
    "This is so frustrating!"
]

print("😊 情緒感知對話測試\n")
for msg in test_messages:
    emotion, confidence = detect_emotion(msg)
    response = generate_empathetic_response(msg, emotion)

    print(f"User: {msg}")
    print(f"  檢測情緒: {emotion} ({confidence:.2%})")
    print(f"Bot: {response}\n")

### 6.2 意圖識別與槽填充

In [ ]:
# 簡單的意圖識別
intent_classifier = pipeline(
    "zero-shot-classification",
    model="facebook/bart-large-mnli"
)

INTENTS = [
    "greeting",
    "weather_query",
    "restaurant_search",
    "general_chat",
    "goodbye"
]

def detect_intent(user_input):
    """
    識別用戶意圖
    """
    result = intent_classifier(user_input, INTENTS)
    return result['labels'][0], result['scores'][0]

def handle_intent(user_input, intent):
    """
    根據意圖處理請求
    """
    if intent == "greeting":
        return "Hello! How can I help you today?"

    elif intent == "weather_query":
        # 實際應串接天氣 API
        return "I'm sorry, I can't check the weather right now. Try asking me about something else!"

    elif intent == "goodbye":
        return "Goodbye! Have a great day!"

    else:
        # 一般閒聊,使用對話模型
        return bot.chat(user_input)


# 測試意圖識別
test_inputs = [
    "Hi there!",
    "What's the weather like today?",
    "Can you recommend a good restaurant?",
    "Tell me a joke.",
    "Goodbye!"
]

print("🎯 意圖識別測試\n")
for inp in test_inputs:
    intent, confidence = detect_intent(inp)
    response = handle_intent(inp, intent)

    print(f"User: {inp}")
    print(f"  意圖: {intent} ({confidence:.2%})")
    print(f"Bot: {response}\n")

## Part 7: 生產部署 (Streamlit App)

### 7.1 Streamlit 聊天應用

In [ ]:
%%writefile chatbot_app.py
# chatbot_app.py - Streamlit 聊天機器人應用

import streamlit as st
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

@st.cache_resource
def load_chatbot_model():
    """載入模型 (僅載入一次)"""
    model_name = "microsoft/DialoGPT-medium"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)
    return tokenizer, model

def generate_response(user_input, chat_history_ids, tokenizer, model):
    """生成回覆"""
    new_input_ids = tokenizer.encode(
        user_input + tokenizer.eos_token,
        return_tensors='pt'
    )

    if chat_history_ids is not None:
        bot_input_ids = torch.cat([chat_history_ids, new_input_ids], dim=-1)
    else:
        bot_input_ids = new_input_ids

    chat_history_ids = model.generate(
        bot_input_ids,
        max_length=1000,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        top_p=0.95,
        temperature=0.7
    )

    response = tokenizer.decode(
        chat_history_ids[:, bot_input_ids.shape[-1]:][0],
        skip_special_tokens=True
    )

    return response, chat_history_ids

# Streamlit UI
st.title("💬 AI 聊天機器人")
st.caption("Powered by DialoGPT (Microsoft)")

# 載入模型
tokenizer, model = load_chatbot_model()

# 初始化對話歷史 (使用 session state)
if "chat_history_ids" not in st.session_state:
    st.session_state.chat_history_ids = None

if "messages" not in st.session_state:
    st.session_state.messages = []

# 顯示對話歷史
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.write(message["content"])

# 用戶輸入
if user_input := st.chat_input("輸入訊息..."):
    # 顯示用戶訊息
    st.session_state.messages.append({"role": "user", "content": user_input})
    with st.chat_message("user"):
        st.write(user_input)

    # 生成回覆
    with st.chat_message("assistant"):
        with st.spinner("思考中..."):
            response, st.session_state.chat_history_ids = generate_response(
                user_input,
                st.session_state.chat_history_ids,
                tokenizer,
                model
            )
            st.write(response)

    # 記錄機器人回覆
    st.session_state.messages.append({"role": "assistant", "content": response})

# 側邊欄: 重置對話
if st.sidebar.button("🔄 重置對話"):
    st.session_state.chat_history_ids = None
    st.session_state.messages = []
    st.rerun()

# 側邊欄: 對話統計
st.sidebar.markdown("### 📊 對話統計")
st.sidebar.metric("對話輪數", len(st.session_state.messages) // 2)

if st.session_state.chat_history_ids is not None:
    st.sidebar.metric("Token 數", st.session_state.chat_history_ids.shape[-1])

### 7.2 運行 Streamlit App

```bash
# 安裝 Streamlit
poetry add streamlit

# 運行應用
poetry run streamlit run chatbot_app.py

# 瀏覽器自動開啟: http://localhost:8501
```

## Part 8: 進階優化

### 8.1 加入記憶功能

In [ ]:
class MemoryEnhancedChatbot:
    """
    帶記憶功能的聊天機器人
    """
    def __init__(self, model_name="microsoft/DialoGPT-medium"):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_pretrained(model_name)
        self.chat_history_ids = None

        # 用戶資訊記憶
        self.user_info = {}

    def extract_user_info(self, user_input):
        """
        從對話中提取用戶資訊
        """
        # 簡單範例: 提取名字
        if "my name is" in user_input.lower():
            name = user_input.lower().split("my name is")[-1].strip().split()[0]
            self.user_info['name'] = name.capitalize()
            print(f"  📝 記住: 用戶名字是 {self.user_info['name']}")

        # 提取其他資訊 (興趣、位置等)
        # 實際應用可使用 NER

    def personalize_response(self, response):
        """
        個性化回覆
        """
        if 'name' in self.user_info:
            # 在回覆中使用用戶名字
            response = response.replace("you", self.user_info['name'], 1)

        return response

    def chat(self, user_input):
        """處理對話並記憶資訊"""
        # 提取用戶資訊
        self.extract_user_info(user_input)

        # 生成回覆 (使用前面定義的函數)
        response, self.chat_history_ids = generate_response(
            user_input,
            self.chat_history_ids
        )

        # 個性化回覆
        response = self.personalize_response(response)

        return response


# 測試記憶功能
memory_bot = MemoryEnhancedChatbot()

dialogue = [
    "Hi! My name is Alice.",
    "What's your favorite color?",
    "Do you remember my name?"
]

print("🧠 記憶功能測試\n")
for inp in dialogue:
    response = memory_bot.chat(inp)
    print(f"User: {inp}")
    print(f"Bot: {response}\n")

## Part 9: 評估與測試

### 9.1 對話品質評估

In [ ]:
def evaluate_response_quality(response, user_input):
    """
    評估回覆品質
    """
    metrics = {}

    # 1. 長度合理性
    metrics['length'] = len(response.split())
    metrics['length_score'] = 1.0 if 5 <= metrics['length'] <= 50 else 0.5

    # 2. 多樣性 (不重複)
    words = response.split()
    unique_ratio = len(set(words)) / len(words) if words else 0
    metrics['diversity_score'] = unique_ratio

    # 3. 相關性 (簡單檢查是否包含用戶輸入關鍵詞)
    user_keywords = set(user_input.lower().split())
    response_keywords = set(response.lower().split())
    overlap = len(user_keywords & response_keywords)
    metrics['relevance_score'] = min(overlap / len(user_keywords), 1.0) if user_keywords else 0

    # 4. 整體分數
    metrics['overall_score'] = (
        metrics['length_score'] * 0.3 +
        metrics['diversity_score'] * 0.4 +
        metrics['relevance_score'] * 0.3
    )

    return metrics


# 評估範例
user_inp = "What do you think about artificial intelligence?"
bot_resp = bot.chat(user_inp)

metrics = evaluate_response_quality(bot_resp, user_inp)

print(f"User: {user_inp}")
print(f"Bot: {bot_resp}\n")
print("品質評估:")
for metric, value in metrics.items():
    print(f"  {metric}: {value:.3f}")

## Part 10: 總結與擴展

### ✅ 本專案完成內容

1. **基礎對話系統**
   - DialoGPT 單輪/多輪對話
   - Blenderbot 整合
   - 對話歷史管理

2. **進階功能**
   - 回覆品質過濾
   - 情緒識別
   - 意圖分類
   - 用戶資訊記憶

3. **生產部署**
   - Streamlit 互動介面
   - 模型快取優化
   - 評估指標

### 🚀 進階擴展方向

#### 功能擴展
- [ ] 整合知識庫 (RAG)
- [ ] 多語言支持
- [ ] 語音輸入/輸出
- [ ] 個性化對話風格

#### 技術優化
- [ ] 使用更大模型 (GPT-3.5, LLaMA)
- [ ] 模型量化加速
- [ ] 多模態對話 (圖片+文字)
- [ ] 強化學習微調 (RLHF)

#### 應用場景
- [ ] 客服機器人
- [ ] 教學助手
- [ ] 心理諮詢機器人
- [ ] 程式碼助手

### 📚 延伸閱讀

- [DialoGPT 論文](https://arxiv.org/abs/1911.00536)
- [Blenderbot 論文](https://arxiv.org/abs/2004.13637)
- [對話系統綜述](https://arxiv.org/abs/2203.08745)
- [Streamlit 文檔](https://docs.streamlit.io/)

---

**專案版本**: v1.0
**建立日期**: 2025-10-17
**作者**: iSpan NLP Team
**授權**: MIT License